## 1-3. 학습 파이프라인
- 5가지 과제(문서분류, 객체명 인식, 질의응답, 문서검색, 문자생성)

In [8]:
!pip install ratsnlp

In [18]:
!pip install nlpbook

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 316.9/316.9 MB 2.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.5.0-py2.py3-none-any.whl size=317425345 sha256=64fbe115dd44bebfa26276182c33f98ebb6873cea440d302ef3d403bac0c7dae
  Stored in directory: /root/.cache/pip/wheels/41/4e/10/c2cf2467f71c678cfc8a6b9ac9241e5e44a01940da8fbb17fc
Successfully built pyspark


1-1. 설정값 선언

- 모델을 만들려면 가장 먼저 각종 설정값을 정해야 합니다. 어떤 프리트레인 모델을 사용할지, 학습에 사용할 데이터는 무엇인지, 학습 결과는 어디에 저장할지 등이 바로 그것입니다. 이 설정값들은 본격적인 학습에 앞서 미리 선언해 둡니다.
- 하이퍼파라미터(hyperparameter) 역시 미리 정해둬야 하는 중요한 정보입니다. 하이퍼파라미터란 모델 구조와 학습 등에 직접 관계된 설정값을 가리킵니다. 예를 들어 러닝 레이트(learning rate), 배치 크기(batch size) 등이 있습니다.

In [19]:
from ratsnlp.nlpbook.classification import ClassificationTrainArguments
args = ClassificationTrainArguments(
    pretrained_model_name="beomi/kcbert-base",
    downstream_corpus_name="nsmc",
    downstream_corpus_root_dir="/content/Korpora",
    downstream_model_dir="/gdrive/My Drive/nlpbook/checkpoint-cls",
    learning_rate=5e-5,
    batch_size=32,
)

1-2. 데이터 다운로드
- downstream_corpus_name(nsmc)에 해당하는 말뭉치를 코랩 환경 로컬의 downstream_corpus_root_dir(/content/Korpora) 아래에 저장

In [20]:
from Korpora import Korpora
Korpora.fetch(
    corpus_name=args.downstream_corpus_name,
    root_dir=args.downstream_corpus_root_dir,
    force_download=True,
)

[nsmc] download ratings_train.txt: 14.6MB [00:00, 44.8MB/s]                            
[nsmc] download ratings_test.txt: 4.90MB [00:00, 26.3MB/s]                           


1-3. KCBert-Base 모델 준비
- 대규모 말뭉치를 활용한 프리트레인에는 많은 리소스가 필요
- BERT, GPT 같은 트랜스포머(Transformer) 계열 모델로 튜토리얼을 진행할 예정인데요. 이 패키지를 쓰면 단 몇 줄만으로 모델을 사용할 수 있습니다.

In [21]:
from transformers import BertConfig, BertForSequenceClassification
pretrained_model_config = BertConfig.from_pretrained(
    args.pretrained_model_name,
    num_labels=2,
)
model = BertForSequenceClassification.from_pretrained(
        args.pretrained_model_name,
        config=pretrained_model_config,
)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:72: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


model.safetensors:   0%|          | 0.00/438M [00:00<?, ?B/s]

Some weights of the model checkpoint at beomi/kcbert-base were not used when initializing BertForSequenceClassification: ['cls.seq_relationship.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at beomi/kcbert-base and are newly i

1-4. 토크나이저 준비하기
- 자연어 처리 모델의 입력은 대개 토큰(token)입니다.
- 여기서 토큰이란 문장(sentence)보다 작은 단위입니다.
- 한 문장은 여러 개의 토큰으로 구성됩니다.
- 토큰 분리 기준은 그때그때 다를 수 있습니다. 문장을 띄어쓰기만으로 나눌 수도 있고, 의미의 최소 단위인 형태소(morpheme) 단위로 분리할 수도 있습니다.

- 문장을 토큰 시퀀스(token sequence)로 분석하는 과정을 토큰화(tokenization), 토큰화를 수행하는 프로그램을 토크나이저(tokenizer)라고 합니다.

In [12]:
from transformers import BertTokenizer
tokenizer = BertTokenizer.from_pretrained(
    args.pretrained_model_name,
    do_lower_case=False,
)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:72: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


vocab.txt:   0%|          | 0.00/250k [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/49.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/619 [00:00<?, ?B/s]

1-5. Data Loader 준비
- 파이토치(PyTorch)는 딥러닝 모델 학습을 지원하는 파이썬 라이브러리입니다. 파이토치에는 데이터 로더(DataLoader)라는 게 포함돼 있습니다. 파이토치로 딥러닝 모델을 만들고자 한다면 이 데이터 로더를 반드시 정의해야 합니다.

- 데이터 로더는 학습 때 데이터를 배치(batch) 단위로 모델에 밀어 넣어주는 역할을 합니다. 전체 데이터 가운데 일부 인스턴스를 뽑아(sample) 배치를 구성합니다. 데이터셋(dataset)은 데이터 로더의 구성 요소 가운데 하나입니다. 데이터셋은 여러 인스턴스(문서+레이블)를 보유하고 있습니다.

- 데이터 로더가 배치를 만들 때 인스턴스를 뽑는 방식은 파이토치 사용자가 자유롭게 정할 수 있습니다.

- 배치는 그 모양이 고정적어야 할 때가 많습니다. 다시 말해 동일한 배치에 있는 문장들의 토큰(input_ids) 개수가 같아야 합니다.

- 이같이 배치의 모양 등을 정비해 모델의 최종 입력으로 만들어주는 과정을 컬레이트(collate)라고 합니다. 컬레이트 과정에는 파이썬 리스트(list)에서 파이토치 텐서(tensor)로의 변환 등 자료형 변환도 포함됩니다. 컬레이트 수행 방식 역시 파이토치 사용자가 자유롭게 구성할 수 있습니다.

In [23]:
from torch.utils.data import DataLoader, RandomSampler
from ratsnlp.nlpbook.classification import NsmcCorpus, ClassificationDataset
corpus = NsmcCorpus()
train_dataset = ClassificationDataset(
    args=args,
    corpus=corpus,
    tokenizer=tokenizer,
    mode="train",
)
train_dataloader = DataLoader(
    train_dataset,
    batch_size=args.batch_size,
    sampler=RandomSampler(train_dataset, replacement=False),
    collate_fn=nlpbook.data_collator,
    drop_last=False,
    num_workers=args.cpu_workers,
)

NameError: name 'nlpbook' is not defined

- 여기서 각 토큰을 그에 해당하는 정수로 변환하는 과정을 인덱싱(indexing)이라고 합니다.
- 인덱싱은 보통 토크나이저가 토큰화와 함께 수행합니다.

1-6. Task 정의
- PyTorch Lightning Model 사용
- PyTorch Lightning: 딥러닝 모델을 학습할 때 반복적인 내용을 대신 수행해줘 사용자가 모델 구축에만 신경쓸 수 있도록 돕는 라이브러리

- 최적화(optimization)란 특정 조건에서 어떤 값이 최대나 최소가 되도록 하는 과정을 가리킵니다.
- 이를 위해 옵티마이저(optimizer), 러닝 레이트 스케줄러(learning rate scheduler) 등을 정의해 둡니다.
- 모델 학습은 배치 단위로 이뤄집니다. 배치를 모델에 입력한 뒤 모델 출력을 정답과 비교해 차이를 계산합니다. 이후 그 차이를 최소화하는 방향으로 모델을 업데이트합니다.
- 이러한 일련의 과정을 스텝(step)이라고 합니다.

1-7. 모델 학습하기
- 트레이너(Trainer)는 파이토치 라이트닝에서 제공하는 객체로 실제 학습을 수행

In [16]:
from ratsnlp.nlpbook.classification import ClassificationTask
task = ClassificationTask(model, args)
trainer = nlpbook.get_trainer(args)
trainer.fit(
    task,
    train_dataloader=train_dataloader,
)

NameError: name 'model' is not defined

## 1-4. 개발환경 설정
- 링크 참고해주세요!
https://ratsgo.github.io/nlpbook/docs/introduction/environment/